#  대화 이력 관리를 위한 메모리 구현(Chat History)

### **학습 목표:**  대화 이력 관리를 위한 메모리 컴포넌트 구현 방법을 실습한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9, 
)

# **채팅 히스토리 관리**

* 채팅봇의 핵심 기능은 이전 대화 내용을 문맥으로 활용하는 것으로, 이를 통해 자연스러운 대화 흐름을 유지할 수 있습니다.

* 가장 기본적인 방식은 이전 메시지들을 모델의 프롬프트에 직접 포함시키는 것이지만, 오래된 메시지는 적절히 제거하여 모델이 처리해야 할 정보량을 조절할 수 있습니다.

* 장시간 진행되는 대화의 경우, 단순히 이전 메시지를 저장하는 것을 넘어 대화 내용을 요약하여 저장하는 등의 고급 메모리 관리 기법을 활용할 수 있습니다.

### 1. **메시지 전달 방식(Message Passing)**

* 메시지 전달 방식은 LangChain에서 가장 기본적인 메모리 구현 방법으로, 이전 대화 기록(chat history)을 체인에 직접 전달하여 문맥을 유지하는 방식입니다.

* 이 방식은 SystemMessage(시스템 지시사항), HumanMessage(사용자 입력), AIMessage(AI 응답) 등 다양한 유형의 메시지를 ChatPromptTemplate을 통해 구조화하며, MessagesPlaceholder를 사용하여 이전 대화 내용을 포함시킵니다.

* 챗봇의 기본적인 메모리 시스템을 구현하는데 사용되며, 이를 통해 AI는 이전 대화 맥락을 이해하고 그에 맞는 적절한 응답을 생성할 수 있습니다.


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ChatPromptTemplate를 사용하여 챗봇의 초기 메시지를 정의
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="messages"),    # 메시지 목록을 동적으로 삽입하는 부분
])

# ChatPromptTemplate에 삽입할 메시지 목록을 정의
messages = [
        HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
        AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
]    

# ChatPromptTemplate에 삽입할 메시지 목록을 업데이트하고 출력 
pprint(prompt.format(messages=messages))

('System: You are a helpful assistant.\n'
 'Human: 안녕하세요. 제 이름은 홍길동입니다.\n'
 'AI: 안녕하세요! 어떻게 도와드릴까요?')


In [5]:
# 대화형 체인을 정의 (prompt -> llm)
chain = prompt | llm

# 기본적인 메시지 전달: 이전 메시지 목록에 새로운 메시지를 추가해서 전달
response = chain.invoke({
    "messages": messages + [HumanMessage(content="제 이름을 기억하나요?")] # 이전 메시지를 기억하는지 확인하는 질문 메시지 추가
})

pprint(response)

AIMessage(content='네, 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 53, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEYkv0bA9pDqfguvq020JkyEyyHVs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e5659eff-d14a-4c40-aecb-2d611c17255c-0', usage_metadata={'input_tokens': 53, 'output_tokens': 18, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


### 2. **RunnableWithMessageHistory**

* RunnableWithMessageHistory는 LangChain에서 대화 기록을 관리하는 고급 기능으로, 체인의 실행 과정에서 메시지 기록을 자동으로 저장하고 검색할 수 있게 해주는 래퍼(wrapper) 클래스입니다.

* 이 기능은 대화 세션별로 독립적인 기록을 유지할 수 있게 해주며, ConfigurableField를 통해 메모리 구성을 유연하게 조정할 수 있습니다. 특히 여러 사용자와 동시에 대화할 때 각 세션의 컨텍스트를 분리하여 관리하는 데 매우 유용합니다.

* 주요 장점은 대화 기록 관리의 자동화와 일관성 있는 메시지 처리이지만, 메모리 저장소 설정과 관리에 추가적인 구성이 필요하다는 점을 고려해야 합니다. 또한 Redis나 다른 외부 저장소와 통합하여 영구적인 대화 기록 보관도 가능합니다.

* 구현 시에는 get_session_history 콜백을 통해 세션ID별로 메시지 기록을 관리하며, 이를 통해 각 대화의 컨텍스트를 정확하게 유지할 수 있습니다.


`(1) 메모리 기반 로컬 저장소 활용`

* `InMemoryHistory` 클래스는 대화 이력의 기본 구조를 제공하며, BaseChatMessageHistory와 BaseModel을 상속받아 메시지를 메모리에서 효율적으로 관리합니다. 특히 messages 리스트를 통해 BaseMessage 객체들을 순차적으로 저장하고, add_messages와 clear 메서드로 히스토리를 유연하게 관리할 수 있습니다.

* 시스템의 핵심인 `store` 변수는 전역 딕셔너리로 구현되어 세션별 대화 이력을 구분하여 저장합니다. session_id를 키로 사용하여 각 세션의 InMemoryHistory 객체에 빠르게 접근할 수 있으며, 이를 통해 다중 사용자 환경에서도 효율적인 대화 관리가 가능합니다.

* `get_session_history` 함수는 세션 관리의 진입점 역할을 하며, 존재하지 않는 세션에 대해 자동으로 새로운 InMemoryHistory 객체를 생성하는 팩토리 패턴을 구현합니다. 이러한 구조를 통해 세션의 생명주기를 자동으로 관리하고 메모리 효율성을 높일 수 있습니다.

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.
2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 고즈넉한 골목길 산책과 카페, 공방 탐방에 좋습니다.
3. 명동 – 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 최적입니다.
4. 남산서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카도 이용해보세요.
5. 홍대 – 젊음의 거리로, 예술과 음악, 다양한 카페와 클럽이 밀집해 있어 활기찬 분위기를 느낄 수 있습니다.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.
7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보내기 좋아요.

필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴게요!


In [8]:
# 대화 히스토리 출력 
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.\n2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 고즈넉한 골목길 산책과 카페, 공방 탐방에 좋습니다.\n3. 명동 – 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 최적입니다.\n4. 남산서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카도 이용해보세요.\n5. 홍대 – 젊음의 거리로, 예술과 음악, 다양한 카페와 클럽이 밀집해 있어 활기찬 분위기를 느낄 수 있습니다.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.\n7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보내기 좋아요.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 310, 'prompt_tokens': 40, 'total_tokens': 350, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-20

In [9]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **명동**, 그리고 **남산서울타워**입니다.

- **경복궁**은 한국의 역사와 문화를 체험할 수 있어 외국인 관광객과 내국인 모두에게 매우 인기가 높습니다. 특히 한복을 입고 방문하면 무료 입장이 가능해 더욱 많은 분들이 찾고 있어요.
- **명동**은 쇼핑과 먹거리의 중심지로, 서울을 방문하는 관광객들이 꼭 들르는 핫플레이스입니다.
- **남산서울타워**는 서울의 전경을 한눈에 볼 수 있는 명소로, 특히 야경이 아름다워 연인들과 가족 단위 방문객에게 인기가 많습니다.

이 세 곳은 서울 여행에서 빠지지 않는 대표적인 인기 명소로 꼽힙니다!


In [10]:
# 대화 히스토리 출력
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복 체험도 인기입니다.\n2. 북촌 한옥마을 – 전통 한옥들이 모여 있는 마을로, 고즈넉한 골목길 산책과 카페, 공방 탐방에 좋습니다.\n3. 명동 – 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 최적입니다.\n4. 남산서울타워 – 서울 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카도 이용해보세요.\n5. 홍대 – 젊음의 거리로, 예술과 음악, 다양한 카페와 클럽이 밀집해 있어 활기찬 분위기를 느낄 수 있습니다.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장까지 즐길 수 있는 복합 문화 공간입니다.\n7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보내기 좋아요.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 310, 'prompt_tokens': 40, 'total_tokens': 350, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-20

`(2) SQLite 데이터베이스 영구 저장소 활용`

* SQLite 통합 구현을 위해서는 먼저 메시지를 저장할 데이터베이스 테이블 구조를 정의하고, BaseChatMessageHistory를 상속받아 메시지 저장/조회 로직을 구현해야 합니다. 이때 세션 ID를 기준으로 대화 내용을 구분하여 관리합니다.

* 시스템 구현 시에는 메시지의 타입(Human/AI), 내용, 메타데이터, 타임스탬프 등의 정보를 체계적으로 저장하고, 필요할 때 효율적으로 검색하고 활용할 수 있도록 적절한 인덱싱 전략을 수립하는 것이 중요합니다.

In [11]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import sqlite3
from typing import List
import json

class SQLiteChatMessageHistory(BaseChatMessageHistory):
    """ 
    SQLite 데이터베이스를 사용하여 챗봇 대화 히스토리를 저장하는 클래스

    Attributes:
        session_id (str): 세션 ID
        db_path (str): SQLite 데이터베이스 파일 경로

    """
    def __init__(self, session_id: str, db_path: str = "chat_history.db"):
        self.session_id = session_id
        self.db_path = db_path
        self._create_tables()
    
    def _create_tables(self):
        """데이터베이스 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 메시지 테이블 생성
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT,
                message_type TEXT,
                content TEXT,
                metadata TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """단일 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO messages (session_id, message_type, content, metadata)
            VALUES (?, ?, ?, ?)
        """, (
            self.session_id,
            message.__class__.__name__,
            message.content,
            json.dumps(message.additional_kwargs)
        ))
        
        conn.commit()
        conn.close()
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """여러 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for message in messages:
            cursor.execute("""
                INSERT INTO messages (session_id, message_type, content, metadata)
                VALUES (?, ?, ?, ?)
            """, (
                self.session_id,
                message.__class__.__name__,
                message.content,
                json.dumps(message.additional_kwargs)
            ))
        
        conn.commit()
        conn.close()
    
    def clear(self) -> None:
        """세션의 모든 메시지 삭제"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM messages WHERE session_id = ?
        """, (self.session_id,))
        
        conn.commit()
        conn.close()
    
    @property
    def messages(self) -> List[BaseMessage]:
        """저장된 메시지 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, metadata
            FROM messages 
            WHERE session_id = ?
            ORDER BY created_at
        """, (self.session_id,))
        
        messages = []
        for row in cursor.fetchall():
            message_type, content, metadata = row
            if message_type == "HumanMessage":
                message = HumanMessage(content=content)
            else:
                message = AIMessage(content=content)
            
            if metadata:
                message.additional_kwargs = json.loads(metadata)
            
            messages.append(message)
        
        conn.close()
        return messages
    

# 세션 ID로 히스토리 가져오기
def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    return SQLiteChatMessageHistory(session_id=session_id)

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "수원에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
수원은 역사와 문화가 풍부한 도시로, 다양한 명소가 있습니다. 수원에서 가볼 만한 곳을 추천해드릴게요.

1. 수원 화성  
- 조선 시대의 성곽으로 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁을 방문해 조선 시대의 건축과 역사를 체험할 수 있습니다.

2. 화성행궁  
- 조선시대 임금이 수원에 머물 때 사용한 행궁으로, 전통 건축물과 다양한 전시가 있습니다. 한복 체험도 가능해 사진 찍기 좋습니다.

3. 수원박물관  
- 수원의 역사와 문화를 알 수 있는 박물관으로, 가족 단위 방문객에게 추천합니다.

4. 광교호수공원  
- 넓은 호수와 산책로가 있어 휴식과 산책을 즐기기에 좋습니다. 주변에 카페와 음식점도 많습니다.

5. 영통 카페거리  
- 다양한 카페와 맛집이 모여 있는 곳으로, 젊은 층에게 인기 있는 장소입니다.

6. 행궁동 벽화마을  
- 예쁜 벽화와 아기자기한 골목길이 있어 사진 찍기 좋은 곳입니다.

수원은 대중교통이 잘 발달되어 있어 이동이 편리합니다. 즐거운 여행 되세요!


In [13]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 문화가 풍부한 도시로, 다양한 명소가 있습니다. 수원에서 가볼 만한 곳을 추천해드릴게요.\n\n1. 수원 화성  \n- 조선 시대의 성곽으로 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁을 방문해 조선 시대의 건축과 역사를 체험할 수 있습니다.\n\n2. 화성행궁  \n- 조선시대 임금이 수원에 머물 때 사용한 행궁으로, 전통 건축물과 다양한 전시가 있습니다. 한복 체험도 가능해 사진 찍기 좋습니다.\n\n3. 수원박물관  \n- 수원의 역사와 문화를 알 수 있는 박물관으로, 가족 단위 방문객에게 추천합니다.\n\n4. 광교호수공원  \n- 넓은 호수와 산책로가 있어 휴식과 산책을 즐기기에 좋습니다. 주변에 카페와 음식점도 많습니다.\n\n5. 영통 카페거리  \n- 다양한 카페와 맛집이 모여 있는 곳으로, 젊은 층에게 인기 있는 장소입니다.\n\n6. 행궁동 벽화마을  \n- 예쁜 벽화와 아기자기한 골목길이 있어 사진 찍기 좋은 곳입니다.\n\n수원은 대중교통이 잘 발달되어 있어 이동이 편리합니다. 즐거운 여행 되세요!', additional_kwargs={'refusal': None}, response_metadata={})]


In [14]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천드린 장소 중 가장 인기 있는 곳은 **수원 화성**입니다.  
수원 화성은 유네스코 세계문화유산으로 지정된 역사적 명소로, 많은 관광객이 방문하는 대표적인 관광지입니다. 성곽을 따라 걷거나 화성행궁을 함께 둘러보며 조선 시대의 건축과 역사를 체험할 수 있어 특히 주말과 휴일에 방문객이 많습니다. 또한 다양한 문화 행사와 야간 조명도 인기 요소 중 하나입니다.  
수원 방문 시 꼭 들러보시길 추천드립니다!


In [15]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 문화가 풍부한 도시로, 다양한 명소가 있습니다. 수원에서 가볼 만한 곳을 추천해드릴게요.\n\n1. 수원 화성  \n- 조선 시대의 성곽으로 유네스코 세계문화유산에 등재되어 있습니다. 성곽을 따라 걷거나 화성행궁을 방문해 조선 시대의 건축과 역사를 체험할 수 있습니다.\n\n2. 화성행궁  \n- 조선시대 임금이 수원에 머물 때 사용한 행궁으로, 전통 건축물과 다양한 전시가 있습니다. 한복 체험도 가능해 사진 찍기 좋습니다.\n\n3. 수원박물관  \n- 수원의 역사와 문화를 알 수 있는 박물관으로, 가족 단위 방문객에게 추천합니다.\n\n4. 광교호수공원  \n- 넓은 호수와 산책로가 있어 휴식과 산책을 즐기기에 좋습니다. 주변에 카페와 음식점도 많습니다.\n\n5. 영통 카페거리  \n- 다양한 카페와 맛집이 모여 있는 곳으로, 젊은 층에게 인기 있는 장소입니다.\n\n6. 행궁동 벽화마을  \n- 예쁜 벽화와 아기자기한 골목길이 있어 사진 찍기 좋은 곳입니다.\n\n수원은 대중교통이 잘 발달되어 있어 이동이 편리합니다. 즐거운 여행 되세요!', additional_kwargs={'refusal': None}, response_metadata={}),
 HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이전에 추천드린 장소 중 가장 인기 있는 곳은 **수원 화성**입니다.  \n수원 화성은 유네스코 세계문화유산으로 지정된 역사적 명소로, 많은 관광객이 방문하는 대표적인 관광지입니다. 성곽을 따라 걷거나 화성행궁을 함께 둘러보며 조선 시대의 건축과 역사를 체험할 수 있어 특히 주말과

In [16]:
# 대화 히스토리 초기화
history.clear()

In [17]:
# 대화 히스토리 출력
pprint(history.messages)

[]


# **메시지 관리 기법**

* 메시지 트리밍은 컨텍스트 윈도우의 토큰 제한을 관리하며, 시스템 메시지 포함 여부와 시작 위치 등을 세밀하게 제어할 수 있습니다.

* 장시간 진행되는 대화의 경우, 이전 대화 내용을 한 문장으로 요약하여 컨텍스트로 활용함으로써 메모리 효율성을 높일 수 있습니다.

* 대화 히스토리가 일정 길이를 초과할 경우, 요약된 내용과 최근 메시지만을 새로운 히스토리로 구성하여 컨텍스트의 품질을 유지하면서도 토큰 사용량을 최적화할 수 있습니다.

### 1. **메시지 트리밍(Message Trimming)**

* `trim_messages` 함수는 컨텍스트 윈도우의 토큰 제한을 관리하기 위한 핵심 도구로, 시스템 메시지를 포함할지 여부와 어디서부터 트리밍을 시작할지 등을 상세하게 설정할 수 있습니다.

* 트리밍 전략으로 "last" 옵션을 사용하면 가장 최근의 메시지부터 시작하여 지정된 토큰 제한에 맞춰 이전 메시지들을 선택적으로 포함시킬 수 있습니다.

In [18]:
from langchain_core.messages import trim_messages

# 메시지 목록을 정의
orginal_messages = [
    HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
    AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
    HumanMessage(content="제 이름을 기억하나요?")
]

# 트리머(trimmer) 생성
# 마지막 메시지 2개만 유지하도록 설정 (token은 메시지를 나타냄) 
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

trimmed_messages = trimmer.invoke(orginal_messages)

pprint(trimmed_messages)

[AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 기억하나요?', additional_kwargs={}, response_metadata={})]


In [19]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field
from typing import List

# 메시지 트리밍이 적용된 인메모리 히스토리 구현
class TrimmedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    max_tokens: int = Field(default=2)  # 유지할 최대 메시지 수
    
    def __init__(self, max_tokens: int = 2, **kwargs):
        """
        TrimmedInMemoryHistory 초기화
        
        Args:
            max_tokens (int): 유지할 최대 메시지 수
            **kwargs: 추가 키워드 인자
        """
        super().__init__(max_tokens=max_tokens, **kwargs)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
        # 메시지 추가 후 트리밍 수행
        trimmer = trim_messages(
            strategy="last",
            max_tokens=self.max_tokens,
            token_counter=len
        )
        self.messages = trimmer.invoke(self.messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 트리밍된 히스토리 가져오기
def get_trimmed_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = TrimmedInMemoryHistory()
    return store[session_id]

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 트리밍된 히스토리 관리 추가
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain,
    get_trimmed_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 한옥 건축과 아름다운 정원을 감상할 수 있어요. 한복 대여 후 방문하면 더 특별한 경험이 됩니다.

2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 옛 서울의 분위기를 느낄 수 있어요.

3. 명동 – 쇼핑과 먹거리가 풍부한 번화가로, 다양한 패션 브랜드와 길거리 음식을 즐길 수 있습니다.

4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 시내 전경을 한눈에 볼 수 있어요. 특히 야경이 아름답습니다.

5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 카페 문화가 발달한 곳이에요. 거리 공연과 독특한 가게들을 구경해보세요.

6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 패션 마켓이 열리는 곳으로, 디자인과 문화에 관심 있다면 추천합니다.

7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보내기 좋아요.

필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴 수 있습니다!


In [20]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 한옥 건축과 아름다운 정원을 감상할 수 있어요. 한복 대여 후 방문하면 더 특별한 경험이 됩니다.\n\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 옛 서울의 분위기를 느낄 수 있어요.\n\n3. 명동 – 쇼핑과 먹거리가 풍부한 번화가로, 다양한 패션 브랜드와 길거리 음식을 즐길 수 있습니다.\n\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 시내 전경을 한눈에 볼 수 있어요. 특히 야경이 아름답습니다.\n\n5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 카페 문화가 발달한 곳이에요. 거리 공연과 독특한 가게들을 구경해보세요.\n\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 패션 마켓이 열리는 곳으로, 디자인과 문화에 관심 있다면 추천합니다.\n\n7. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보내기 좋아요.\n\n필요하시면 각 장소별 교통편이나 추천 일정도 알려드릴 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 327, 'prompt_tokens': 40, 'total_tokens': 367, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens'

In [21]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.

- **경복궁**은 한국의 역사와 문화를 대표하는 궁궐로, 외국인 관광객뿐만 아니라 내국인에게도 매우 인기 있는 명소입니다. 특히 한복을 입고 방문하면 사진 찍기 좋은 장소로 유명해요.

- **N서울타워**는 서울의 랜드마크 중 하나로, 서울 전경을 한눈에 볼 수 있어 관광객들이 많이 찾는 곳입니다. 특히 저녁 시간대의 야경이 아름다워 데이트 코스나 가족 나들이 장소로 인기가 많습니다.

두 곳 모두 서울을 처음 방문하는 분들께 꼭 추천드리는 명소입니다!


In [22]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.\n\n- **경복궁**은 한국의 역사와 문화를 대표하는 궁궐로, 외국인 관광객뿐만 아니라 내국인에게도 매우 인기 있는 명소입니다. 특히 한복을 입고 방문하면 사진 찍기 좋은 장소로 유명해요.\n\n- **N서울타워**는 서울의 랜드마크 중 하나로, 서울 전경을 한눈에 볼 수 있어 관광객들이 많이 찾는 곳입니다. 특히 저녁 시간대의 야경이 아름다워 데이트 코스나 가족 나들이 장소로 인기가 많습니다.\n\n두 곳 모두 서울을 처음 방문하는 분들께 꼭 추천드리는 명소입니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 391, 'total_tokens': 561, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEYlNQVSMI6s9lr2MtQWAwnlOnCwM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--89

In [23]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이 장소의 정식 명칭은 무엇인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
네, 말씀드린 두 장소의 정식 명칭은 다음과 같습니다.

- 경복궁: **경복궁(景福宮, Gyeongbokgung Palace)**
- N서울타워: **N서울타워(N Seoul Tower)** 또는 공식 명칭은 **서울타워(Seoul Tower)**입니다.

경복궁은 조선 시대의 대표적인 궁궐이며, N서울타워는 남산 위에 위치한 통신 및 전망 타워로 서울의 상징적인 랜드마크입니다.


In [24]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이 장소의 정식 명칭은 무엇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 말씀드린 두 장소의 정식 명칭은 다음과 같습니다.\n\n- 경복궁: **경복궁(景福宮, Gyeongbokgung Palace)**\n- N서울타워: **N서울타워(N Seoul Tower)** 또는 공식 명칭은 **서울타워(Seoul Tower)**입니다.\n\n경복궁은 조선 시대의 대표적인 궁궐이며, N서울타워는 남산 위에 위치한 통신 및 전망 타워로 서울의 상징적인 랜드마크입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 235, 'total_tokens': 350, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_ed35aa4f5f', 'id': 'chatcmpl-CEYlQY2wjy4PkUTM3k7WHasUbTQJI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b3fd86af-e572-49c2-bf43-883f631ce47c-0', usage_metadata={'input_tokens': 235, 'output_tokens': 115, 'total_tokens

### 2. **대화 요약 저장**

* 대화가 길어질 경우, 전체 대화 내용을 요약하여 컨텍스트로 활용하는 방식으로 이전 대화의 핵심을 추출합니다.

* 일반적으로 메시지 히스토리가 지정된 길이(예: 4개의 메시지)를 초과할 경우, 이전 대화들을 요약하고 가장 최근의 메시지만 유지하는 방식으로 새로운 대화 히스토리를 구성합니다.

* 이러한 요약 메모리 방식을 통해 토큰 사용량을 크게 줄이면서도 대화의 핵심 문맥을 유지할 수 있으며, 특히 장시간 진행되는 대화에서 효과적입니다.

In [25]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import List
    
class SummarizedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    summary_threshold: int = Field(default=6)  # 요약을 시작할 메시지 수
    # 요약에 사용할 LLM - lambda를 사용하여 필드가 초기화될 때마다 새로운 ChatOpenAI 인스턴스 생성
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, top_p=0.9))  
    
    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        self.messages.extend(new_messages)

        print(f"메시지 수: {len(self.messages)}")
        
        # 메시지 수가 임계값을 넘으면 요약 수행
        if len(self.messages) >= self.summary_threshold:
            # 마지막 사용자 메시지 저장 (HumanMessage, AIMessage 순서로 생성)
            last_user_message = self.messages[-2]
            last_ai_message = self.messages[-1]
            
            # 요약 생성
            summary_prompt = (
                "Distill the above chat messages into a single summary message. "
                "Include as many specific details as you can."
                "Use the original language and tone of the conversation."
            )
            
            summary_chain_messages = [
                SystemMessage(content=(
                    "You are a helpful assistant. "
                    "Your task is to summarize the conversation accurately."
                )),
                *self.messages[:-2],  # 마지막 대화 턴을 제외한 모든 메시지
                HumanMessage(content=summary_prompt)
            ]
            
            # 요약 생성
            summary = self.llm.invoke(summary_chain_messages)
            
            # 메시지 리스트 초기화 후 요약과 마지막 메시지 추가
            self.messages = [
                summary,
                last_user_message,
                last_ai_message
            ]
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 요약된 히스토리 가져오기
def get_summarized_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = SummarizedInMemoryHistory()
    return store[session_id]


# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 체인 구성
chain_with_summarized_history = RunnableWithMessageHistory(
    chain,
    get_summarized_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(user_1)를 사용하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 2
여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.

2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 정취를 느껴보세요. 카페와 공방도 많아 여유롭게 산책하기 좋아요.

3. 명동 – 쇼핑과 먹거리가 풍부한 번화가입니다. 화장품, 패션 아이템을 쇼핑하고 길거리 음식도 즐겨보세요.

4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있어요. 특히 야경이 아름답습니다.

5. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 패션 마켓이 열리는 곳으로 트렌디한 분위기를 느낄 수 있습니다.

6. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 다양한 야외 활동을 즐길 수 있어요.

필요하시면 맛집 추천이나 교통 정보도 알려드릴게요!


In [26]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.\n\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 정취를 느껴보세요. 카페와 공방도 많아 여유롭게 산책하기 좋아요.\n\n3. 명동 – 쇼핑과 먹거리가 풍부한 번화가입니다. 화장품, 패션 아이템을 쇼핑하고 길거리 음식도 즐겨보세요.\n\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있어요. 특히 야경이 아름답습니다.\n\n5. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 패션 마켓이 열리는 곳으로 트렌디한 분위기를 느낄 수 있습니다.\n\n6. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 다양한 야외 활동을 즐길 수 있어요.\n\n필요하시면 맛집 추천이나 교통 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_tokens': 40, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-

In [27]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 4
여행 가이드 답변:
이전에 추천드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **N서울타워**입니다.

- **경복궁**은 한국의 대표적인 역사 유적지로, 한국 전통문화를 체험하고 사진 찍기 좋은 명소라 외국인 관광객과 내국인 모두에게 매우 인기가 많아요.

- **N서울타워**는 서울의 랜드마크로, 특히 야경 명소로 유명해 연인들이 많이 찾는 장소입니다. 전망대에서 서울 시내를 한눈에 볼 수 있어 방문객이 꾸준히 많습니다.

두 곳 모두 서울 여행에서 꼭 방문해야 할 인기 명소로 꼽히니 일정에 참고해 주세요!


In [28]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.\n\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 정취를 느껴보세요. 카페와 공방도 많아 여유롭게 산책하기 좋아요.\n\n3. 명동 – 쇼핑과 먹거리가 풍부한 번화가입니다. 화장품, 패션 아이템을 쇼핑하고 길거리 음식도 즐겨보세요.\n\n4. N서울타워 – 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있어요. 특히 야경이 아름답습니다.\n\n5. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 패션 마켓이 열리는 곳으로 트렌디한 분위기를 느낄 수 있습니다.\n\n6. 한강공원 – 한강을 따라 조성된 공원으로, 자전거 타기, 피크닉, 야경 감상 등 다양한 야외 활동을 즐길 수 있어요.\n\n필요하시면 맛집 추천이나 교통 정보도 알려드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_tokens': 40, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-

In [29]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "전주에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 6
여행 가이드 답변:
전주에서 가볼 만한 곳을 추천해드릴게요!

1. 전주한옥마을 – 전통 한옥이 잘 보존된 마을로, 한국 전통 문화와 건축을 체험할 수 있어요. 한복 대여도 가능해 사진 찍기 좋습니다.

2. 경기전 – 조선 태조 이성계의 어진을 모신 곳으로, 역사와 전통을 느낄 수 있는 중요한 문화재입니다.

3. 전동성당 – 고딕 양식의 아름다운 성당으로, 전주한옥마을 근처에 있어 함께 방문하기 좋아요.

4. 남부시장 – 전주의 대표 재래시장으로, 다양한 먹거리와 특산품을 즐길 수 있습니다. 특히 전주비빔밥과 한옥마을 길거리 음식이 유명해요.

5. 덕진공원 – 호수와 정원이 어우러진 공원으로, 산책과 휴식을 즐기기에 좋은 장소입니다.

6. 전주향교 – 조선시대 교육기관으로, 고즈넉한 분위기 속에서 전통 문화를 체험할 수 있어요.

전주는 맛집도 유명하니 전주비빔밥, 콩나물국밥, 전주식 한정식도 꼭 맛보시길 추천드립니다! 필요하시면 맛집 정보도 알려드릴게요.


In [30]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 곳으로 경복궁, 북촌 한옥마을, 명동, N서울타워, 동대문 디자인 플라자(DDP), 한강공원을 추천드렸습니다. 그중 가장 인기 있는 곳은 경복궁과 N서울타워인데요, 경복궁은 한국 전통문화 체험과 사진 명소로 내외국인 모두에게 사랑받고, N서울타워는 서울 야경 명소이자 연인들의 데이트 코스로 유명해 꾸준히 방문객이 많습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 556, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEYliYYfKbEwJGHkVaXsINKR8AtrJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d10b4869-6677-4bbc-8c7f-9454ffd6fa35-0', usage_metadata={'input_tokens': 556, 'output_tokens': 120, 'total_tokens': 676, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),

In [31]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 서로 비슷한 장소가 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 5
여행 가이드 답변:
서울과 전주에서 비슷한 성격의 장소를 비교해보면 다음과 같습니다:

1. 전통 한옥마을  
- 서울: 북촌 한옥마을 — 전통 한옥이 모여 있는 마을로, 한국 전통문화 체험과 산책에 좋습니다.  
- 전주: 전주한옥마을 — 규모가 더 크고 전통 한옥이 잘 보존되어 있으며, 한복 체험과 전통 음식도 즐길 수 있습니다.

2. 역사적 궁궐/문화재  
- 서울: 경복궁 — 조선시대 대표 궁궐로, 한국 전통 건축과 역사를 체험할 수 있는 곳입니다.  
- 전주: 경기전 — 조선 태조 이성계의 어진을 모신 곳으로, 역사적 의미가 깊은 문화재입니다.

3. 전통 시장  
- 서울: 남대문시장, 광장시장 등 — 다양한 먹거리와 쇼핑을 즐길 수 있는 전통 시장입니다.  
- 전주: 남부시장 — 전주 특산물과 길거리 음식을 즐길 수 있는 재래시장입니다.

4. 종교 건축물  
- 서울: 조계사, 명동성당 등 — 역사적이고 아름다운 사찰과 성당이 있습니다.  
- 전주: 전동성당 — 고딕 양식의 아름다운 성당으로 전통과 현대가 어우러진 장소입니다.

두 도시 모두 전통과 현대가 조화를 이루는 관광지가 많아, 각각의 특색을 느끼면서 비슷한 유형의 장소를 즐기실 수 있습니다.


In [32]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 곳으로 경복궁, 북촌 한옥마을, 명동, N서울타워, 동대문 디자인 플라자(DDP), 한강공원을 추천드렸습니다. 그중 가장 인기 있는 곳은 경복궁과 N서울타워인데요, 경복궁은 한국 전통문화 체험과 사진 명소로 내외국인 모두에게 사랑받고, N서울타워는 서울 야경 명소이자 연인들의 데이트 코스로 유명해 꾸준히 방문객이 많습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 556, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEYliYYfKbEwJGHkVaXsINKR8AtrJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d10b4869-6677-4bbc-8c7f-9454ffd6fa35-0', usage_metadata={'input_tokens': 556, 'output_tokens': 120, 'total_tokens': 676, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),

In [33]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 각각 한 장소만 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 7
여행 가이드 답변:
서울에서는 경복궁을 추천드립니다. 조선시대 대표 궁궐로서 한국 전통 건축과 역사를 깊이 체험할 수 있고, 광화문 광장과도 가까워 주변 관광도 편리합니다.

전주에서는 전주한옥마을을 추천드립니다. 전통 한옥이 잘 보존된 마을로, 한복 체험과 전통 음식, 다양한 문화 행사를 즐길 수 있어 전주의 매력을 한눈에 느낄 수 있는 명소입니다.


In [34]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 곳으로 경복궁, 북촌 한옥마을, 명동, N서울타워, 동대문 디자인 플라자(DDP), 한강공원을 추천드렸고, 특히 경복궁과 N서울타워가 인기 명소입니다. 전주에서는 전주한옥마을, 경기전, 전동성당, 남부시장, 덕진공원, 전주향교를 추천하며, 전주한옥마을은 한복 체험과 전통 음식으로 유명합니다. 서울과 전주에서 비슷한 장소로는 전통 한옥마을(서울 북촌 한옥마을 vs. 전주한옥마을), 역사적 궁궐/문화재(서울 경복궁 vs. 전주 경기전), 전통 시장(서울 남대문시장 등 vs. 전주 남부시장), 종교 건축물(서울 명동성당 등 vs. 전주 전동성당)이 있어 두 도시 모두 전통과 현대가 조화를 이루는 관광지를 즐길 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 231, 'prompt_tokens': 881, 'total_tokens': 1112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEYlqM0wLRq7dbvVaxwWNUL5SqdOI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e919b0d-96ab-477e-9c32-60dd01638140-0', usage_metadata={'inp

In [35]:
# 대화 히스토리 초기화
history.clear()

In [36]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[]


---
# **[실습]**

- 메시지 트리밍과 대화 요약 저장을 결합하여 메시지를 관리하는 기능을 구현합니다. 

1. 새로운 클래스 구조

In [ ]:
class TrimmedAndSummarizedHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage]
    max_tokens: int  # 트리밍 기준
    summary_threshold: int  # 요약 기준
    llm: ChatOpenAI
    summarized_messages: List[BaseMessage]  # 요약된 메시지 저장용

2. 메시지 처리 로직
- 새 메시지 추가시 max_tokens 체크
- 트리밍 발생하면 제거될 메시지 식별
- 제거 예정 메시지들은 요약하여 summarized_messages에 저장
- 현재 메시지는 트리밍된 상태로 유지

3. 요약 프로세스
- 트리밍으로 제거될 메시지들만 선별
- 선별된 메시지들에 대해 summary_chain 실행
- 요약본을 시스템 메시지로 변환하여 저장

In [104]:
# 여기에 코드를 작성하세요.